# CS109A:  Project Milestone 2 - Group 13

### Project 5 - Housing Crisis & Homelessness

Group 13: Andrew Russo, John Wu, Ryan Han, Vivian Lu

In [174]:
import pandas as pd
import numpy as np
import os

data_path = os.path.join(os.getcwd(),'Data')
pd.options.display.max_columns = None

<hr style="height:2pt">

### Current Question: 

By recent estimates, over 600,000 Americans live in some state of homelessness each year. In recent decades, the quality of available data on homelessness in the United States has greatly improved, but there is still a long way to go. In this project you will work to visualize trends and create models to understand the top predictors of homelessness. Some questions you may persue are: 

**Have the top predictors changed over time? What events or policies took place that led to a change in the intensity of homelessness (either positive or negative)?** 

Data at the national level should be incorporated in some way even if you decided to focus on a specific city such as LA, NYC, or Boston. 

### Current Data Sources:

https://www.kaggle.com/schirmerchad/bostonhoustingmlnd <br/>
https://www.kaggle.com/adamschroeder/homelessness-comparison-between-states/data <br/>
https://www.kaggle.com/umerkk12/homelessness-in-us

### Additional Data Sources (Per Vivian):
https://www.hudexchange.info/resource/3031/pit-and-hic-data-since-2007/

<hr style="height:2pt">

Description of the Data:
1. What type of data are you dealing with?
2. What methods have you used to explore the data (initial explorations, data cleaning, etc)?

Abbreviations:
- HIC = Housing Inventory Count
- PIT = Point in Time
- ES = Emergency Shelter 
- TH = Transitional Housing 
- SH = Safe Haven
- RRH = Rapid Re-Housing
- PSH = Permanent Supportive Housing
- OPH = Other Permenanent Housing 

In [175]:
# Class to help with cleaning dataframes

class worksheets:
    
    # lists of features to parse
    
    race_list = ['White','Black or African American',
                 'American Indian or Alaska Native','Multiple Races',
                 'Asian','Native Hawaiian or Other Pacific Islander']
    eth_list = ['Hispanic/Latino','Non-Hispanic/Non-Latino']
    gender_list = ['Transgender','Gender Non-Conforming','Male','Female']
    age_list = ['Under 18','Age 18 to 24','Over 24']
    homeless_type = ['ES','SH','TH','RRH','PSH','OPH']
    group_type =['Individuals','Families','Unaccompanied Youth (Under 25)','Children of Parenting Youth',
                 'Parenting Youth Under 18','Parenting Youth Age 18-24']
    count_list = ['Overall','Unsheltered','Sheltered']


    def __init__(self,data, name):
        self.data = data
        self.name = name
        self.feature_lists = [race_list, eth_list,
                              gender_list, age_list,
                              homeless_type, group_type,
                              count_list]
        self.feature_names = ['Race','Ethnicity',
                              'Gender','Age_Group',
                              'Shelter_Type','Group_Type',
                              'Count_Type']

    def feat_parser(self, var, l_values, upper=True):
         if upper == True:
            var = var.upper()
            for feature in l_values:
                feature = feature.upper()
                if feature in var:
                    return feature
            return None
         else:
            for feature in l_values:
                if feature in var:
                    return feature
            return None
    
    def to_feature(self):
        data = self.data
        for feature, feat_list in zip(self.feature_names, self.feature_lists):
            if feature == 'Shelter_Type':
                upper = False
            data[feature] = [self.feat_parser(var,feat_list) for var in data['variable']]
        self.data = data
        
        
# helper function to read in data

def read_data(data_file, type_of_table):
    if type_of_table == 'ST':
        index_col = ['State','Number of CoCs', 'Year']
        total_col = 'State'
        skipfooter = 1
    else:
        index_col = ['CoC Number','CoC Name','Year']
        total_col ='CoC Name'
    wb = pd.ExcelFile(data_file)
    sheets = wb.sheet_names
    df = []
    for sheet in sheets:
           if sheet != 'Revisions' and sheet != 'CoC Mergers':
            #print(sheet)  # Used for debugging
            df_sheet = pd.read_excel(data_file, sheet_name = sheet)
            df_sheet['Year'] = sheet

            df_sheet = df_sheet.drop(df_sheet.filter(regex='Total').columns, axis=1) # dropping total columns since these would double count observations
            df_sheet = df_sheet[df_sheet[total_col]!= 'Total'] #dropping total 

            keep_cols = set(index_col)
            melt_cols = list(set(list(df_sheet.columns)) - keep_cols)

            df_sheet = pd.melt(df_sheet, id_vars=index_col,value_vars = melt_cols)
            df.append(df_sheet)
    return pd.concat(df)

In [176]:
# data_files
PIT_ST_file = data_path+'/2007-2021-PIT-Counts-by-State.xlsx'
PIT_CoC_file = data_path+'/2007-2021-PIT-Counts-by-CoC.xlsx'


# reading files
PIT_ST = read_data(PIT_ST_file,'ST')
PIT_CoC = read_data(PIT_CoC_file,'CoC')
PIT_CoC.dropna(subset='CoC Name',inplace=True) # dropping extra footers and totals

In [177]:
# adding features

PIT_ST = worksheets(PIT_ST,'ST')
PIT_ST.to_feature()

PIT_CoC = worksheets(PIT_CoC, 'CoC')
PIT_CoC.to_feature()

In [182]:
# outputting data
PIT_ST_df = PIT_ST.data
PIT_ST_df.to_excel('PIT_ST_Cleaned.xlsx')

PIT_CoC_df = PIT_CoC.data
PIT_CoC_df.to_excel('PIT_CoC_Cleaned.xlsx')

ValueError: This sheet is too large! Your sheet size is: 1139039, 12 Max sheet size is: 1048576, 16384

Visualizations and Captions that summarize the noteworthy findings of the exploratory data analysis. 

In [179]:
# Code here

Revised project question based on your group's interests and the insights gained through EDA.  Be sure to keep the scope manageable.

In [180]:
# Code here

A baseline mode.  (implemented or clearly described)

* baseline model should be more sophisticated than a "naive" model like in HW5 that simply predicts the majority class or mean response value.  Through class proportions (if dealing with a classification problem) is importnat information to repore in your EDA as this will almost certainly inform later modeling choices. 

(HW5 Naive model = if we simply apply the same outcome to all applicants based on the proportion)

In [181]:
# Code here

2 or more references (papers, articles, etc.) that your groups thinks will be relevant to guiding your project work. These can be related to methods and/or the problem domain of the project.

**John**:
- National Alliance to end homelessness (endhomelessness.org)
   - Provides additional dataset between 2007 to 2021.  (COVID data from 2021 to 2022 is more incomplete)
   - Adds HUD dataset.  
   
- 